In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import torchvision
import torchvision.transforms as transforms

import os
import argparse

import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
print(torch.__version__) 
print(torch.cuda.is_available())
! nvidia-smi

In [ ]:
# Training
def train(epoch, net, criterion, trainloader, scheduler):
    device = 'cuda'
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(trainloader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

        if (batch_idx+1) % 50 == 0:
          print("iteration : %3d, loss : %0.4f, accuracy : %2.2f" % (batch_idx+1, train_loss/(batch_idx+1), 100.*correct/total))

    scheduler.step()
    return train_loss/(batch_idx+1), 100.*correct/total

def test(epoch, net, criterion, testloader):
    device = 'cuda'
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.inference_mode():
        for batch_idx, (inputs, targets) in enumerate(testloader):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    return test_loss/(batch_idx+1), 100.*correct/total

In [ ]:
# defining resnet models

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes, stride=1):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.conv3 = nn.Conv2d(planes, self.expansion *
                               planes, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm2d(self.expansion*planes)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_planes != self.expansion*planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(self.expansion*planes)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = F.relu(self.bn2(self.conv2(out)))
        out = self.bn3(self.conv3(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # This is the "stem"
        # For CIFAR (32x32 images), it does not perform downsampling
        # It should downsample for ImageNet
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        # four stages with three downsampling
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512*block.expansion, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

In [ ]:
# these are commonly used data augmentations
# random cropping and random horizontal flip
# lastly, we normalize each channel into zero mean and unit standard deviation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    torchvision.transforms.RandomErasing(p=0.8, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=(0.4914, 0.4822, 0.4465))
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

trainset = torchvision.datasets.CIFAR10(
    root='./data', train=True, download=True, transform=transform_train)
trainloader = torch.utils.data.DataLoader(
    trainset, batch_size=128, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(
    root='./data', train=False, download=True, transform=transform_test)

# we can use a larger batch size during test, because we do not save 
# intermediate variables for gradient computation, which leaves more memory
testloader = torch.utils.data.DataLoader(
    testset, batch_size=256, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer',
           'dog', 'frog', 'horse', 'ship', 'truck')

#### Training

In [ ]:
# main body
config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 0
}


net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])
#scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 15, gamma=1, last_epoch=-1)

train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(1, 5):
    train_loss_, train_acc_ = train(epoch, net, criterion, trainloader, scheduler)
    test_loss_, test_acc_ = test(epoch, net, criterion, testloader)
    
    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, test loss " + \
      ": %0.4f, test accuracy : %2.2f") % (epoch, train_loss_, train_acc_, test_loss_, test_acc_))
    
    train_loss.append(train_loss_)
    test_loss.append(test_loss_)
    train_acc.append(train_acc_)
    test_acc.append(test_acc_)

In [ ]:
torch.save({
            'epoch': epoch,
            'model_state_dict': net.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': train_loss_,
            'acc': train_acc_,
            }, './checkpoint/test.pth')

In [ ]:
model = ResNet18().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])

checkpoint = torch.load('./checkpoint/test.pth')
model.load_state_dict(checkpoint['model_state_dict'])
optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
epoch = checkpoint['epoch']
loss = checkpoint['loss']

train(epoch, model, criterion, trainloader, scheduler)

#### Save Images

In [ ]:
plt.figure(dpi=600)
plt.plot(range(len(train_loss)), train_loss, 'b')
plt.plot(range(len(test_loss)), test_loss, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Learning Rate = 0.1: Loss vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/loss_11.png", dpi=600)
plt.show()

plt.figure(dpi=600)
plt.plot(range(len(train_acc)), train_acc, 'b')
plt.plot(range(len(test_acc)), test_acc, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Learning Rate = 0.1: Accuracy vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/acc_11.png", dpi=600)
plt.show()

In [ ]:
def save_checkpoint(net, acc, epoch):
    # Save checkpoint.
    print('Saving..')
    state = {
        'net': net.state_dict(),
        'acc': acc,
        'epoch': epoch,
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/const.pth')
    
save_checkpoint(net, test_acc, epoch)

In [ ]:
# main body
config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4
}

net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(1, 301):
    train_loss_, train_acc_ = train(epoch, net, criterion, trainloader, scheduler)
    test_loss_, test_acc_ = test(epoch, net, criterion, testloader)
    
    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, test loss " + \
      ": %0.4f, test accuracy : %2.2f") % (epoch, train_loss_, train_acc_, test_loss_, test_acc_))
    
    train_loss.append(train_loss_)
    test_loss.append(test_loss_)
    train_acc.append(train_acc_)
    test_acc.append(test_acc_)
   
plt.figure(dpi=600)
plt.plot(range(len(train_loss)), train_loss, 'b')
plt.plot(range(len(test_loss)), test_loss, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Weight Decay->0.0005: Loss vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/loss_5e-4.png", dpi=600)
plt.show()

plt.figure(dpi=600)
plt.plot(range(len(train_acc)), train_acc, 'b')
plt.plot(range(len(test_acc)), test_acc, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Weight Decay->0.0005: Accuracy vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/acc_5e-4.png", dpi=600)
plt.show()
    
    
config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 1e-2
}

net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(1, 301):
    train_loss_, train_acc_ = train(epoch, net, criterion, trainloader, scheduler)
    test_loss_, test_acc_ = test(epoch, net, criterion, testloader)
    
    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, test loss " + \
      ": %0.4f, test accuracy : %2.2f") % (epoch, train_loss_, train_acc_, test_loss_, test_acc_))
    
    train_loss.append(train_loss_)
    test_loss.append(test_loss_)
    train_acc.append(train_acc_)
    test_acc.append(test_acc_)
    
plt.figure(dpi=600)
plt.plot(range(len(train_loss)), train_loss, 'b')
plt.plot(range(len(test_loss)), test_loss, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Weight Decay->0.01: Loss vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/loss_1e-2.png", dpi=600)
plt.show()

plt.figure(dpi=600)
plt.plot(range(len(train_acc)), train_acc, 'b')
plt.plot(range(len(test_acc)), test_acc, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Weight Decay->0.01: Accuracy vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/acc_1e-2.png", dpi=600)
plt.show()

In [ ]:
# main body
config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4
}


net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(1, 301):
    train_loss_, train_acc_ = train(epoch, net, criterion, trainloader, scheduler)
    test_loss_, test_acc_ = test(epoch, net, criterion, testloader)
    
    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, test loss " + \
      ": %0.4f, test accuracy : %2.2f") % (epoch, train_loss_, train_acc_, test_loss_, test_acc_))
    
    train_loss.append(train_loss_)
    test_loss.append(test_loss_)
    train_acc.append(train_acc_)
    test_acc.append(test_acc_)

In [ ]:
plt.figure(dpi=600)
plt.plot(range(len(train_loss)), train_loss, 'b')
plt.plot(range(len(test_loss)), test_loss, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Cutout Augmentation 30%: Loss vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/loss_aug_1.png", dpi=600)
plt.show()

plt.figure(dpi=600)
plt.plot(range(len(train_acc)), train_acc, 'b')
plt.plot(range(len(test_acc)), test_acc, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Cutout Augmentation 30%: Accuracy vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/acc_aug_1.png", dpi=600)
plt.show()

In [ ]:
# main body
config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4
}


net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(1, 301):
    train_loss_, train_acc_ = train(epoch, net, criterion, trainloader, scheduler)
    test_loss_, test_acc_ = test(epoch, net, criterion, testloader)
    
    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, test loss " + \
      ": %0.4f, test accuracy : %2.2f") % (epoch, train_loss_, train_acc_, test_loss_, test_acc_))
    
    train_loss.append(train_loss_)
    test_loss.append(test_loss_)
    train_acc.append(train_acc_)
    test_acc.append(test_acc_)

In [ ]:
plt.figure(dpi=600)
plt.plot(range(len(train_loss)), train_loss, 'b')
plt.plot(range(len(test_loss)), test_loss, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Cutout Augmentation 50%: Loss vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/loss_aug_2.png", dpi=600)
plt.show()

plt.figure(dpi=600)
plt.plot(range(len(train_acc)), train_acc, 'b')
plt.plot(range(len(test_acc)), test_acc, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Cutout Augmentation 50%: Accuracy vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/acc_aug_2.png", dpi=600)
plt.show()

In [ ]:
# main body
config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4
}


net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(1, 301):
    train_loss_, train_acc_ = train(epoch, net, criterion, trainloader, scheduler)
    test_loss_, test_acc_ = test(epoch, net, criterion, testloader)
    
    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, test loss " + \
      ": %0.4f, test accuracy : %2.2f") % (epoch, train_loss_, train_acc_, test_loss_, test_acc_))
    
    train_loss.append(train_loss_)
    test_loss.append(test_loss_)
    train_acc.append(train_acc_)
    test_acc.append(test_acc_)

In [ ]:
plt.figure(dpi=600)
plt.plot(range(len(train_loss)), train_loss, 'b')
plt.plot(range(len(test_loss)), test_loss, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Cutout Augmentation 100%: Loss vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/loss_aug_3.png", dpi=600)
plt.show()

plt.figure(dpi=600)
plt.plot(range(len(train_acc)), train_acc, 'b')
plt.plot(range(len(test_acc)), test_acc, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Cutout Augmentation 100%: Accuracy vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./Output/acc_aug_3.png", dpi=600)
plt.show()

In [ ]:
# main body
config = {
    'lr': 0.01,
    'momentum': 0.9,
    'weight_decay': 5e-4
}


net = ResNet18().to('cuda')
criterion = nn.CrossEntropyLoss().to('cuda')
optimizer = optim.SGD(net.parameters(), lr=config['lr'],
                      momentum=config['momentum'], weight_decay=config['weight_decay'])
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=300)

train_loss = []
train_acc = []
test_loss = []
test_acc = []

for epoch in range(1, 301):
    train_loss_, train_acc_ = train(epoch, net, criterion, trainloader, scheduler)
    test_loss_, test_acc_ = test(epoch, net, criterion, testloader)
    
    print(("Epoch : %3d, training loss : %0.4f, training accuracy : %2.2f, test loss " + \
      ": %0.4f, test accuracy : %2.2f") % (epoch, train_loss_, train_acc_, test_loss_, test_acc_))
    
    train_loss.append(train_loss_)
    test_loss.append(test_loss_)
    train_acc.append(train_acc_)
    test_acc.append(test_acc_)

In [ ]:
plt.figure(dpi=600)
plt.plot(range(len(train_loss)), train_loss, 'b')
plt.plot(range(len(test_loss)), test_loss, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Loss")
plt.title("Cutout Augmentation 80%: Loss vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./loss_aug_4.png", dpi=600)
plt.show()

plt.figure(dpi=600)
plt.plot(range(len(train_acc)), train_acc, 'b')
plt.plot(range(len(test_acc)), test_acc, 'r')
plt.xlabel("Number of epochs")
plt.ylabel("Accuracy")
plt.title("Cutout Augmentation 80%: Accuracy vs Number of epochs")
plt.legend(['train', 'test'])
plt.savefig("./acc_aug_4.png", dpi=600)
plt.show()